In [2]:
import pandas as pd
import numpy as np

In [3]:
title_akas_df = pd.read_csv('title.akas.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
episode_df = pd.read_csv('title.episode.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
basics_df = pd.read_csv('title.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
ratings_df = pd.read_csv('title.ratings.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
crew_df = pd.read_csv('title.crew.tsv.gz',compression='gzip', header=0, sep='\t', quotechar='"')
principals_df = pd.read_csv('title.principals.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
name_basic_df = pd.read_csv('name.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
print("reading done")


/var/folders/mj/mqkdjsmx093601gfdjss778r0000gn/T/ipykernel_37406/2497871281.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_akas_df = pd.read_csv('title.akas.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')
/var/folders/mj/mqkdjsmx093601gfdjss778r0000gn/T/ipykernel_37406/2497871281.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv('title.basics.tsv.gz', compression='gzip', header=0, sep='\t', quotechar='"')


reading done


In [4]:
# create DIRECTOR CSV
name_basic_db_df = name_basic_df[~name_basic_df['primaryProfession'].isna()]
director_db_df = name_basic_db_df[name_basic_db_df['primaryProfession'].str.contains('director')]
director_db_df = director_db_df[['nconst', 'primaryName', 'birthYear', 'deathYear']]
director_db_df = director_db_df[~(director_db_df['birthYear'] == '\\N')]
director_db_df['deathYear'] = director_db_df['deathYear'].replace('\\N', np.nan)
director_db_df.to_csv('director.csv', index=False)
print("director done")
director_db_df

director done


,nconst,primaryName,birthYear,deathYear
4,nm0000005,Ingmar Bergman,1918,2007
7,nm0000008,Marlon Brando,1924,2004
9,nm0000010,James Cagney,1899,1986
18,nm0000019,Federico Fellini,1920,1993
23,nm0000024,John Gielgud,1904,2000
...,...,...,...,...
12592197,nm9987048,Michael Maltby,1942,NaN
12595760,nm9990940,Emma-Marie Fusco,2001,NaN
12597170,nm9992905,Hepi Mita,1986,NaN
12597202,nm9992946,Christoph Blaschke,1993,NaN


In [5]:
# create Actor CSV
name_basic_db_df = name_basic_df[~name_basic_df['primaryProfession'].isna()]
actors_db_df = name_basic_db_df[(name_basic_db_df['primaryProfession'].str.contains('actor')) | (name_basic_db_df['primaryProfession'].str.contains('actress'))]
actors_db_df = actors_db_df[~(actors_db_df['birthYear'] == '\\N')]
actors_db_df = actors_db_df[['nconst', 'primaryName', 'birthYear', 'deathYear']]
actors_db_df = actors_db_df[actors_db_df['primaryName'].str.len() < 100]
actors_db_df['deathYear'] = actors_db_df['deathYear'].replace('\\N', np.nan)
actors_db_df.to_csv('actors.csv', index=False)
print("actor done")
actors_db_df

actor done


,nconst,primaryName,birthYear,deathYear
0,nm0000001,Fred Astaire,1899,1987
1,nm0000002,Lauren Bacall,1924,2014
2,nm0000003,Brigitte Bardot,1934,NaN
3,nm0000004,John Belushi,1949,1982
4,nm0000005,Ingmar Bergman,1918,2007
...,...,...,...,...
12597098,nm9992812,Daisy,1994,NaN
12597426,nm9993199,Denis Nurulin,1997,NaN
12597591,nm9993379,Fanny Hozleiter,1988,NaN
12597727,nm9993535,Henry Lawfull,2006,NaN


In [6]:
# create MOVIE CSV
movie_db_df = basics_df[basics_df['titleType'] == 'movie']
movie_db_df = pd.merge(movie_db_df, ratings_df, on='tconst', how='inner')
movie_db_df = movie_db_df.drop_duplicates()
director_df = principals_df[principals_df['category'] == 'director'][['tconst', 'nconst']]
movie_db_df = pd.merge(movie_db_df, director_df, on='tconst', how='inner')
movie_db_df.drop_duplicates(subset ="tconst",keep = 'first', inplace = True)
movie_db_df = movie_db_df.drop_duplicates()
movie_db_df = movie_db_df[~(movie_db_df['genres'] == '\\N')]
movie_db_df = movie_db_df[~(movie_db_df['runtimeMinutes'] == '\\N')]
movie_db_df = movie_db_df[~(movie_db_df['startYear'] == '\\N')]
movie_db_df = movie_db_df[['tconst', 'averageRating', 'numVotes', 'startYear', 'runtimeMinutes', 'primaryTitle', 'nconst']]
movie_db_df = movie_db_df[movie_db_df['primaryTitle'].str.len() < 100]
movie_db_df = movie_db_df[movie_db_df['nconst'].isin(director_db_df['nconst'].unique())]
movie_db_df.to_csv('movie.csv', index=False)
print("movie done")
movie_db_df

movie done


,tconst,averageRating,numVotes,startYear,runtimeMinutes,primaryTitle,nconst
0,tt0000009,5.3,206,1894,45,Miss Jerry,nm0085156
1,tt0000147,5.3,474,1897,100,The Corbett-Fitzsimmons Fight,nm0714557
3,tt0000574,6.0,831,1906,70,The Story of the Kelly Gang,nm0846879
4,tt0000591,4.4,20,1907,90,The Prodigal Son,nm0141150
8,tt0000679,5.1,68,1908,120,The Fairylogue and Radio-Plays,nm0091767
...,...,...,...,...,...,...,...
304038,tt9913084,6.7,51,2019,75,Diabolik sono io,nm0812850
304042,tt9914254,7.1,8,2019,85,A Cherry Tale,nm1375922
304052,tt9915946,6.8,11,1993,96,Nuestra película,nm0652213
304056,tt9916270,5.8,1422,2020,84,Il talento del calabrone,nm1480867


In [7]:
# create plays_in_movies CSV
plays_in_movie_db_df = principals_df[principals_df['tconst'].isin(movie_db_df['tconst'].unique())]
plays_in_movie_db_df = plays_in_movie_db_df[plays_in_movie_db_df['category'].isin(['actor','actress'])]
plays_in_movie_db_df = plays_in_movie_db_df[['tconst', 'nconst']]
plays_in_movie_db_df = plays_in_movie_db_df.drop_duplicates()
plays_in_movie_db_df = plays_in_movie_db_df[plays_in_movie_db_df['nconst'].isin(actors_db_df['nconst'].unique())]
plays_in_movie_db_df.to_csv('plays_in_movies.csv', index=False)
print("plays in done")
plays_in_movie_db_df

plays in done


,tconst,nconst
24,tt0000009,nm0063086
25,tt0000009,nm0183823
26,tt0000009,nm1309758
1043,tt0000574,nm0846887
1044,tt0000574,nm0846894
...,...,...
56577991,tt9916270,nm3080119
56577992,tt9916270,nm2063290
56578176,tt9916362,nm3766704
56578177,tt9916362,nm0107165


In [8]:
# create genre_movies CSV
movie_list_db_df = basics_df[basics_df['titleType'] == 'movie']
movie_genre_db_df = movie_list_db_df.join(movie_list_db_df['genres'].str.split(',', expand=True).rename(columns={0:'Genre1', 1:'Genre2', 2:'Genre3'}))
movie_genre_db_df = movie_genre_db_df[['tconst', 'Genre1', 'Genre2', 'Genre3']]
movie_genre_db_df = movie_genre_db_df[~(movie_genre_db_df['Genre1'] == '\\N')]
pd.melt(movie_genre_db_df, id_vars='tconst', value_vars=['Genre1','Genre2','Genre3'], value_name='Genre')
movie_genre_db_df = movie_genre_db_df.melt('tconst', value_name='Genre').drop(columns='variable')
movie_genre_db_df = movie_genre_db_df[~movie_genre_db_df['Genre'].isna()]
movie_genre_db_df = movie_genre_db_df[movie_genre_db_df['tconst'].isin(movie_db_df['tconst'].unique())]
movie_genre_db_df.to_csv('genre_movies.csv', index=False)
print("genre done")
movie_genre_db_df


genre done


,tconst,Genre
0,tt0000009,Romance
1,tt0000147,Documentary
2,tt0000574,Action
3,tt0000591,Drama
7,tt0000679,Adventure
...,...,...
1725927,tt9891874,Thriller
1726077,tt9898858,Crime
1726105,tt9900060,Crime
1726124,tt9900782,Crime
